In [1]:
import numpy as np
import pyproj

Plan is to update simulation to solve in ECEF coordinates and output lla with noise to model GPS.
The sim will start with intial lla, and convert to initial ECEF, then solve in ECEF. Finally, output to nav using ecef2lla.

In [2]:
# ECEF position
x = 652954.1006
y = 4774619.7919
z =-4167647.7937

pos = np.array([x,y,z])

# Expected lla 
# lat  -41.0445318235 deg
# lon   82.2128095674 deg
# alt 2274.39966936

In [3]:
ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')

In [4]:
ecef_2_lla = pyproj.Transformer.from_crs(
    {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
    {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},
    )
lla_2_ecef = pyproj.Transformer.from_crs(
    {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},
    {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
    )
lon2, lat2, alt2 = ecef_2_lla.transform(x,y,z,radians=False)
x2,y2,z2 = lla_2_ecef.transform(lon2, lat2, alt2,radians=False)

print(f"lat = {lat2}\nlon = {lon2}\nalt = {alt2}")
print(f"x   ={x2}\ny   ={y2}\nz   ={z2}")

lat = -41.04453182346411
lon = 82.2128095673836
alt = 2274.399669399485
x   =652954.1005999997
y   =4774619.791899999
z   =-4167647.7937000585


In [5]:
def ecef2lla(pos):
    transformer = pyproj.Transformer.from_crs(
        {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
        {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},
        )
    lon,lat,alt = transformer.transform(pos[0],pos[1],pos[2],
                                        radians=False)
    return np.array([lat,lon,alt])

def lla2ecef(lla):
    transformer = pyproj.Transformer.from_crs(
        {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},
        {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
        )
    # lla (lat, lon, alt)
    # transform() expects lon, lat, alt
    x,y,z = transformer.transform(lla[1],lla[0],lla[2],
                                  radians=False)
    return np.array([x,y,z])

    

In [6]:
lla = ecef2lla(pos)
pos = lla2ecef(lla)
lla2= ecef2lla(pos)
print(lla)
print(pos)
print(lla2)

[ -41.04453182   82.21280957 2274.3996694 ]
[  652954.1006      4774619.7919     -4167647.79370006]
[ -41.04453182   82.21280957 2274.39966944]


# Process in sim

In [7]:
# Initial LLA
lat = 33
lon = -110
alt = 1200 # ft
lla = np.array([lat,lon,alt])
ecef = lla2ecef(lla)
print(f"Starting Position")
print(f"  lla = {lla}")
print(f"  ecef = {ecef}")

# Gps Model Output
gpsNoise = 1
gpsPos_L = ecef + gpsNoise * np.random.randn(3)
gpsLLA = ecef2lla(gpsPos_L)
print(f"GPS Position")
print(f"  gpsPos_L = {gpsPos_L}")
print(f"  gpsLLA   = {gpsLLA}")

# NavData - Convert LLA to ECEF prior to EKF
navLLA = gpsLLA
navEcef = lla2ecef(navLLA)
print("Nav Position - EKF input")
print(f"  navEcef = {navEcef}")



Starting Position
  lla = [  33 -110 1200]
  ecef = [-1831682.44471769 -5032506.15647328  3454612.20801992]
GPS Position
  gpsPos_L = [-1831683.11871175 -5032505.95410848  3454612.77318759]
  gpsLLA   = [  33.00000407 -110.00000752 1200.3416602 ]
Nav Position - EKF input
  navEcef = [-1831683.11871175 -5032505.95410848  3454612.7731876 ]


# GeodeticModel

In [8]:
import numpy as np
from numpy import sin, cos, sqrt, degrees, radians, arctan, arctan2

class GeodeticModel():

    # WGS84 Model constants
    a = 6378137 # (m) Semimajor Axis
    b = 6356752.3142 # (m) Semiminor Axis
    esq = 6.69437999014 * 0.001 # First Eccentricity Squared
    e1sq = 6.73949674228 * 0.001 # Second Eccentricity Squared
    f = 1 / 298.257223563 # Flattening


    def __init__(self, lla0):

        lat0, lon0, alt0 = lla0

        # Save initial lla
        self._lla0 = lla0
        self._ecef0 = self.geodetic2ecef(lla0)

        # ECEF/NED transformation matrices
        ecef_x, ecef_y, ecef_z = self._ecef0
        phiP = arctan2( ecef_z,
                sqrt( ecef_x*ecef_x + ecef_y*ecef_y )
                )

        self.C_toLfromECEF = self.calcR0(phiP, lon0)
        self.C_toECEFfromL = self.calcR0(lat0, lon0).T

    def calcR0(self,lat,lon):
        sLat = sin( lat )
        cLat = cos( lat )
        sLon = sin( lon )
        cLon = cos( lon )

        R0 = np.array([
            [-sLat*cLon, -sLon, cLat*cLon],
            [-sLat*sLon,  cLon, cLat*sLon],
            [      cLat,     0,      sLat],
            ])
        return R0


    def geodetic2ecef(self,lla, wantDeg=False):
        """
        Convert geodetic (WGS84) coordeinates to ECEF using
        World Geodetic System 1984 (WGS84) model

        lat, lon in radians
        alt in meters

        """
        lat,lon,alt = lla

        if wantDeg:
            lat = radians(lat)
            lon = radians(lon)

        xi = sqrt(1 - self.esq * sin(lat)*sin(lat))
        x = (self.a / xi + alt) * cos(lat) * cos(lon)
        y = (self.a / xi + alt) * cos(lat) * sin(lon)
        z = (self.a / xi * (1-self.esq) + alt) * sin(lat)

        return np.array([x, y, z])

    def ecef2geodetic(self, ecef, wantDeg=False):
        """
        Convert ECEF coordinates to geodetic.
        J. Zhu, "Conversion of Earth-centered Earth-fixed coordinates
        to geodetic coordinates," IEEE Transactions on Aerospace and
        Electronic Systems, vol. 30, pp. 957-961, 1994.
        """
        x,y,z = ecef

        r = sqrt(x*x + y*y)
        Esq = self.a*self.a - self.b*self.b
        F = 54 * self.b*self.b * z*z
        G = r*r + (1-self.esq) * z*z - self.esq*Esq
        C = (self.esq*self.esq * F * r*r) / (G**3)
        S = np.cbrt(1 + C + sqrt(C*C + 2*C))
        P = F / (3 * (S + 1 / S + 1)**2 * G*G)
        Q = np.sqrt(1 + 2 * self.esq*self.esq * P)
        r_0 =  -(P * self.esq * r) / (1+Q) + np.sqrt(0.5 * self.a*self.a*(1 + 1.0 / Q) - \
            P * (1-self.esq) * z*z / (Q * (1+Q)) - 0.5 * P * r*r)
        t = (r - self.esq * r_0)**2
        U = np.sqrt(t + z*z)
        V = np.sqrt(t + (1-self.esq) * z*z)
        Z_0 = self.b*self.b * z / (self.a * V)
        alt = U * (1 - self.b*self.b / (self.a * V))
        lat = arctan((z + self.e1sq * Z_0) / r)
        lon = arctan2(y, x)

        if wantDeg:
            lat = degrees(lat)
            lon = degrees(lon)

        return np.array([lat, lon, alt])

    def ecef2ned(self,ecef):
        return np.matmul(self.C_toLfromECEF, ecef - self._ecef0)

    def ned2ecef(self,ned):
        return np.matmul(self.C_toECEFfromL, ned) + self._ecef0

    def geodetic2ned(self,lla):
        return self.ecef2ned( self.geodetic2ecef( lla ) )

    def ned2geodetic(self,ned):
        return self.ecef2geodetic( self.ned2ecef( ned ) )


In [10]:
RAD2DEG = 180.0/np.pi
DEG2RAD = 1/RAD2DEG

# Initial LLA
lat = 33.333 * DEG2RAD
lon = -110 * DEG2RAD
alt = 400 # m
lla = np.array([lat,lon,alt])
ecef = lla2ecef(lla)
print(f"PyProj")
print(f"  lla = {lla}")
print(f"  ecef = {ecef}")


gm = GeodeticModel(lla*0.99999)
ecef = gm.geodetic2ecef(lla)
ned = gm.ecef2ned(ecef)
ecef2 = gm.ned2ecef(ned)
lla2 = gm.ecef2geodetic(ecef2)

print(f"Geodetic Model")
print(f"  lla    = {lla}")
print(f"  ecef   = {ecef}")
print(f"  ned    = {ned}")
print(f"  ecef2  = {ecef2}")
print(f"  lla2   = {lla2}")
print(f"  llaErr = {(lla2-lla)/lla*100}")



PyProj
  lla = [  0.5817706   -1.91986218 400.        ]
  ecef = [6374630.0604708  -213680.54782304   64331.84101747]
Geodetic Model
  lla    = [  0.5817706   -1.91986218 400.        ]
  ecef   = [-1824542.15595058 -5012888.37431726  3485093.74129518]
  ned    = [ 25.30524509 -88.70253218 -57.85959881]
  ecef2  = [-1824542.25065727 -5012888.37431726  3485093.46617459]
  lla2   = [  0.58177056  -1.91986219 399.87597306]
  llaErr = [-6.69857640e-06  8.68948866e-07 -3.10067348e-02]
